In [1]:
import numpy as np

In [2]:
n = 3
low = 0
high = 1
card = 2

In [3]:
def sample_utility(n, low = 0, high = 1):
    xs = [None] * n
    ys = [None] * n
    for i in range(n):
        xs[i] = np.random.uniform(low,high)
        ys[i] = np.random.uniform(low,high)
    return xs,ys

In [4]:
def partition_region(k, low = 0, high = 1):
    s = [None] * k * k
    cnt = 0
    for i in range(k):
        for j in range(k):
            s[cnt] = [[i/k, j/k], [(i+1)/k, (j+1)/k]]
            cnt = cnt + 1
    return s

In [5]:
# region: has the form of [[0.0, 0.0], [0.3333333333333333, 0.3333333333333333]]
def check_eligibility(region, x, y):
    if x >= region[0][0] and x <= region[1][0]:
        return True
    return False

In [6]:
# tailored to card = 2
def best_response(n, eligible_set, xs, ys, card = 2):
    filtered_set_ind = []
    for i in range(n):
        for j in range(n):
            for region in eligible_set:
                if check_eligibility(region, xs[i], ys[i]) and check_eligibility(region, xs[j], ys[j]):
                    filtered_set_ind.append((i,j))
                break
    top = 0
    top_ind = None
    for ind in filtered_set_ind:
        cur = ys[ind[0]] + ys[ind[1]]
        if cur >= top:
            top = cur
            top_ind = ind
    return top, top_ind

In [7]:
print(partition_region(3, 0, 1))

[[[0.0, 0.0], [0.3333333333333333, 0.3333333333333333]], [[0.0, 0.3333333333333333], [0.3333333333333333, 0.6666666666666666]], [[0.0, 0.6666666666666666], [0.3333333333333333, 1.0]], [[0.3333333333333333, 0.0], [0.6666666666666666, 0.3333333333333333]], [[0.3333333333333333, 0.3333333333333333], [0.6666666666666666, 0.6666666666666666]], [[0.3333333333333333, 0.6666666666666666], [0.6666666666666666, 1.0]], [[0.6666666666666666, 0.0], [1.0, 0.3333333333333333]], [[0.6666666666666666, 0.3333333333333333], [1.0, 0.6666666666666666]], [[0.6666666666666666, 0.6666666666666666], [1.0, 1.0]]]


In [8]:
# generating a collection of symmetric eligible sets which are the same for every basis
def generate_eligible_set(k, partition):
    collections = [None] * 2 ** (k ** 2)
    for i in range(2 ** (k ** 2)):
        rem = []
        collections[i] = []
        num = i
        for j in range(k ** 2):
            if num % 2 == 1:
                collections[i].append(partition[j])
            num = num // 2
            if num == 0:
                break
    return collections

In [9]:
partition = partition_region(3, 0, 1)

In [10]:
eligible_sets = generate_eligible_set(3, partition)

In [11]:
xs, ys = sample_utility(3, 0, 1)
print(xs, ys)


[0.15892981489879987, 0.009241726871821454, 0.5128455946924487] [0.6858314129933423, 0.49397302072186133, 0.4702774316174394]


In [12]:
def find_optimal_set(n, eligible_sets, xs, ys):
    top = 0
    top_set = None
    for es in eligible_sets:
        br = best_response(3, es, xs, ys, 2)
        if br[0] >= top:
            top = br[0]
            top_set = es
    return top, top_set

In [14]:
for trial in range(500):
    xs, ys = sample_utility(3,0,1)
    find_optimal_set(3, eligible_sets, xs, ys)

In [ ]:
xs, ys = sample_utility(5,0,1)
partition = partition_region(5, 0, 1)
eligible_sets = generate_eligible_set(5, partition)
find_optimal_set(5, eligible_sets, xs, ys)

In [15]:
import numpy as np

# Function to check if a point is in an eligible part
def is_in_eligible_set(x, y, eligible_parts):
    grid_x = int(x * k)
    grid_y = int(y * k)
    return eligible_parts[grid_x, grid_y]

# Local search procedure
def local_search(n, k, num_trials, num_iterations, X, Y, bases):
    # Initialize eligible parts
    eligible_parts = np.random.choice([0, 1], (k, k))
    best_performance = evaluate_performance(eligible_parts, num_trials, X, Y, bases)
    
    for _ in range(num_iterations):
        if np.random.rand() < 0.5:  # Coin toss
            if np.any(eligible_parts == 0):
                # Find a part that is not in the set (is 0) to switch to 1
                while True:
                    i, j = np.random.randint(0, k, size=2)
                    if eligible_parts[i, j] == 0:
                        eligible_parts[i, j] = 1
                        break
            else:
                # Skip this iteration if no parts can be added
                continue
        else:
            # Check if it's possible to remove a part (if not all are 0)
            if np.any(eligible_parts == 1):
                # Find a part that is in the set (is 1) to switch to 0
                while True:
                    i, j = np.random.randint(0, k, size=2)
                    if eligible_parts[i, j] == 1:
                        eligible_parts[i, j] = 0
                        break
        
        # Evaluate new performance
        new_performance = evaluate_performance(eligible_parts, num_trials, X, Y, bases)
        if new_performance < best_performance:
            # Revert changes if performance decreased
            eligible_parts[i, j] = 1 - eligible_parts[i, j]
        else:
            best_performance = new_performance
    
    return eligible_parts, best_performance

# Evaluate performance of the eligible set
def evaluate_performance(eligible_parts, num_trials, X, Y, bases):
    total_principal_utility = 0
    for trial in range(num_trials):
        trial_utility = 0
        for basis in bases:
            if is_in_eligible_set(X[basis[0], trial], X[basis[1], trial], eligible_parts):
                trial_utility += Y[basis[0], trial] + Y[basis[1], trial]
        total_principal_utility += trial_utility
    return total_principal_utility / num_trials

# Parameters
n = 3  # Number of solutions
k = 5  # Grid size for the partition (k x k)
num_trials = 50  # Number of trials to compute average utility
num_iterations = 20  # Number of iterations for local search

# Utilities
X = np.random.uniform(0, 1, (n, num_trials))
Y = np.random.uniform(0, 1, (n, num_trials))

# Define the bases for a 2-uniform matroid over {1, 2, 3}
bases = [(0, 1), (0, 2), (1, 2)]

# Run local search and print the result
final_eligible_set, final_performance = local_search(n, k, num_trials, num_iterations, X, Y, bases)
print("Final Eligible Set (grid shown as 0s and 1s):")
print(final_eligible_set)
print("Final Performance:", final_performance)

Final Eligible Set (grid shown as 0s and 1s):
[[1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 1 1]]
Final Performance: 2.605921636234421


In [18]:
from concurrent.futures import ProcessPoolExecutor
import time


def run_exp(n = 3,k = 5, num_trials = 10, num_iterations = 30, seed = None):
    # Utilities
    if seed is not None:
        np.random.seed(seed)
    X = np.random.uniform(0, 1, (n, num_trials))
    Y = np.random.uniform(0, 1, (n, num_trials))
#     print("X={0},Y={1}".format(X,Y))

    # Define the bases for a 2-uniform matroid over {1, 2, 3}
    bases = [(0, 1), (0, 2), (1, 2)]

    # Run local search and print the result
    final_eligible_set, final_performance = local_search(n, k, num_trials, num_iterations, X, Y, bases)
    return f"Final Eligible Set (grid shown as 0s and 1s):\n{final_eligible_set}\nFinal Performance: {final_performance}"

# Running experiments in parallel
# Running experiments in parallel
num_experiments = 20
n = 3
k = 5
trial = 50
iteration = 10
with ProcessPoolExecutor() as executor:
    # Prepare arguments for each experiment if needed differently, use zip if parameters vary
    n_values = [n] * num_experiments
    k_values = [k] * num_experiments
    trials_values = [trial] * num_experiments
    iterations_values = [iteration] * num_experiments
    seeds = [int(time.time()) + i for i in range(num_experiments)]

    results = executor.map(run_exp, n_values, k_values, trials_values, iterations_values, seeds)

    for result in results:
        print(result)

Final Eligible Set (grid shown as 0s and 1s):
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]]
Final Performance: 2.9319698182579232
Final Eligible Set (grid shown as 0s and 1s):
[[1 1 0 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]]
Final Performance: 2.7320381473208153
Final Eligible Set (grid shown as 0s and 1s):
[[1 0 1 1 0]
 [1 0 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 1 1 1 1]]
Final Performance: 2.420501161582987
Final Eligible Set (grid shown as 0s and 1s):
[[1 1 1 1 1]
 [1 1 0 0 1]
 [1 1 1 1 1]
 [1 0 1 0 1]
 [1 1 0 0 1]]
Final Performance: 2.1635992734940417
Final Eligible Set (grid shown as 0s and 1s):
[[0 1 1 1 1]
 [1 1 0 1 1]
 [0 0 1 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]]
Final Performance: 2.258033113431763
Final Eligible Set (grid shown as 0s and 1s):
[[1 1 1 1 0]
 [1 0 1 0 1]
 [1 0 1 1 0]
 [1 1 0 1 0]
 [1 1 1 1 1]]
Final Performance: 1.9286196300575036
Final Eligible Set (grid shown as 0s and 1s):
[[1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 